In [1]:
#!/usr/bin/python


'''
rightSphnix:
* create Labels and store to disk
* build models
'''

##################################################################################
#########################       Configure       ##################################
##################################################################################

assets = 'Test'   # Typically AllStocks, SchwabOneSource, SchwabETFs, or Test
horizon = 2       # prediction horizon in days

totalBuildTimeAllowed_seconds = 28800


startDate = '2010-01-01'



In [2]:

##################################################################################
###########################       Imports       ##################################
##################################################################################
print('importing packages')
from multiprocessing import Pool
import pandas
import os
import uyulala
#reload(uyulala)

import datetime
import numpy
import random
import string
import subprocess
import time
from psutil import virtual_memory
import shutil
from pathlib import Path
import math
import glob
import shutil

totMem = virtual_memory().total
availMem = virtual_memory().available

folderName = 'Assets-'+assets+'--Hrzn-'+str(horizon)


importing packages


In [3]:

##################################################################################
#################              Clear directories          ########################
##################################################################################
print('clearing directories')
try:
    shutil.rmtree(os.path.join(uyulala.dataDir,'labeled',folderName))
    os.makedirs(os.path.join(uyulala.dataDir,'labeled',folderName))
except:
    os.makedirs(os.path.join(uyulala.dataDir,'labeled',folderName))

try:
    shutil.rmtree(os.path.join(uyulala.modelsDir,folderName))
    os.makedirs(os.path.join(uyulala.modelsDir,folderName))
except:
    os.makedirs(os.path.join(uyulala.modelsDir,folderName))


clearing directories


In [4]:
'''
##################################################################################
################# Get and transform data (run leftSphnix) ########################
##################################################################################
print('getting and transforming data')
if assets!="Test":
    import warnings
    warnings.filterwarnings("ignore")


filePath = os.path.join(uyulala.uyulalaDir,'greatRiddleGate','leftSphnix.py')
print('making call: '+'python %s --assets=%s --horizon=%i --start=%s' % (filePath,assets,horizon,startDate))
subprocess.call('python %s --assets=%s --horizon=%i --start=%s' % (filePath,assets,horizon,startDate), shell=True)

'''

getting and transforming data
making call: python /Users/Damian/Documents/uyulala/greatRiddleGate/leftSphnix.py --assets=Test --horizon=2 --start=2010-01-01


0

In [5]:

##################################################################################
########################       Create        ###############################
##################################################################################
print('creating ')

evaluate = [ f.replace('.csv','') for f in os.listdir(os.path.join(uyulala.dataDir,'raw',folderName)) if f.endswith(".csv") ]


creating 


In [6]:

def createLabels(asset=''):
    try:
        labeled = pandas.read_csv(os.path.join(uyulala.dataDir,'raw',folderName,asset+'.csv'),parse_dates=['DateCol']).set_index('DateCol',drop=False)
        labeled = labeled.drop_duplicates(subset=['Date'], keep='last') # KEEP EVERYTHING BELOW THIS POINT IN ORDER
        print('label for biggest loss') # Key Regression Field (what's the biggest loss?)
        labeled = uyulala.lowPercentChange(df=labeled,horizon=horizon)
        print('label for highest gain') # Key Regression Field (what's the predicted return?)
        labeled = uyulala.percentChange(df=labeled,horizon=horizon,HighOrClose='High')
        print('label for whether higest gain comes before biggest loss') # Key Classification Field (is it a good buy?)
        labeled = uyulala.expectedReturnPct(df=labeled,horizon=horizon)
        print('add weights column')  #add weights
        labeled = uyulala.weights(df=labeled, horizon=horizon,weightForIncrease=1,weightForDecrease=2)
        labeled = labeled.drop(['Open','High','Low','Close','Volume'],axis=1)
        labeled['YearMo'] = labeled['DateCol'].dt.strftime('%Y%m')
        labeled['Asset'] = labeled['Symbol']
        labeled.to_parquet(os.path.join(uyulala.dataDir,'labeled',folderName),index=False,partition_cols=['YearMo','Asset'])
        return asset
    except:
        print('unable to create label for '+asset)
        pass




In [ ]:

print('labelling data')
for i in range(0,len(evaluate),400):
    l = evaluate[i:i+400]
    pool = Pool(uyulala.availableCores,maxtasksperchild=1)
    pool.map(createLabels, l)
    pool.close()
    pool.join()

print('Done labelling data')


labelling data
label for biggest losslabel for biggest losslabel for biggest losslabel for biggest losslabel for biggest loss




label for highest gainlabel for highest gainlabel for highest gainlabel for highest gainlabel for highest gain



In [3]:

##################################################################################
##########################       Load Data       #################################
##################################################################################
try: h2o.cluster().shutdown()
except: pass

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.frame import H2OFrame

try:
    h2o.init(nthreads = -1,max_mem_size="%sG" % int(totMem/1500000000/1.5),min_mem_size="%sG" % int(availMem/1500000000/1.5))
except:
    time.sleep(20)
    h2o.init(nthreads = -1,max_mem_size="%sG" % int(totMem/1500000000/1.5),min_mem_size="%sG" % int(availMem/1500000000/1.5))



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Users/Damian/opt/anaconda3/envs/uyulala/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/5b/s4769fcn60d842cy18f7nc3h0000gn/T/tmpi0gfopzl
  JVM stdout: /var/folders/5b/s4769fcn60d842cy18f7nc3h0000gn/T/tmpi0gfopzl/h2o_Damian_started_from_python.out
  JVM stderr: /var/folders/5b/s4769fcn60d842cy18f7nc3h0000gn/T/tmpi0gfopzl/h2o_Damian_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_Damian_feic16
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.223 Gb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
### IMPORT DATA FOR HYPERPARAMETER TUNING
YrMos = [str(f)[-6:] for f in Path(os.path.join(uyulala.dataDir,'labeled',folderName)).glob('**/*') if f.is_dir()]
YrMos = [f for f in YrMos if f.startswith('20') | f.startswith('19')]
ootMonths = YrMos[-int(len(YrMos)*0.1):] #use latest 10% of months as holdout set

dataSize = sum(f.stat().st_size for f in Path(os.path.join(uyulala.dataDir,'transformed',folderName)).glob('**/*') if f.is_file() \
               & ~any([x in str(f) for x in ['YearMo=' + sub for sub in ootMonths] ]) ) \
            + sum(f.stat().st_size for f in Path(os.path.join(uyulala.dataDir,'labeled',folderName)).glob('**/*') if f.is_file() \
               & ~any([x in str(f) for x in ['YearMo=' + sub for sub in ootMonths] ]) )
ratio = ((availMem/2000000000) / (20.0000000000000)) / (dataSize/1000000000)
print('full data size: {}gb'.format(dataSize/1000000000.00))

transformed_files = ['YearMo='+str(f).split('YearMo=')[1] for f in Path(os.path.join(uyulala.dataDir,'transformed',folderName)).glob('**/*') if f.is_file() \
               & ~any([x in str(f) for x in ['YearMo=' + sub for sub in ootMonths] ]) ]

if ratio < 0.98:
    print('reducing file size by {}%'.format(100*(1-ratio)))
    k=math.ceil(len(transformed_files)*ratio)
    sampledFiles=random.choices(transformed_files, k=max(1,min(k,len(transformed_files))))
else:
    sampledFiles=transformed_files
#print('Files to use: {}'.format(sampledFiles))

full data size: 0.061800569gb


In [5]:
fullDF = h2o.import_file(path=os.path.join(uyulala.dataDir,'transformed',folderName),pattern = "(%s)" % ('|'.join(sampledFiles),),col_types={'DateCol':'enum','Date':'enum'}).na_omit().merge(h2o.import_file(path=os.path.join(uyulala.dataDir,'labeled',folderName),pattern = ".*\.parquet",col_types={'DateCol':'enum','Date':'enum'}).na_omit()).na_omit()
fullDF

Parse progress: |█████████████████████████████████████████████████████████| 100%


/Users/Damian/opt/anaconda3/envs/uyulala/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: ParseError at file nfs://Users/Damian/Documents/uyulala/data/transformed/Assets-Test--Hrzn-2/YearMo=201001/Asset=AAPL/22a11e605b1d47c3a1e9a89b35987348.parquet; error = 'Unsupported type override (Numeric -> Enum). Column DateCol will be parsed as Numeric'
  warnings.warn(w)


Parse progress: |█████████████████████████████████████████████████████████| 100%


/Users/Damian/opt/anaconda3/envs/uyulala/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: ParseError at file nfs://Users/Damian/Documents/uyulala/data/labeled/Assets-Test--Hrzn-2/YearMo=201001/Asset=AAPL/04dd8b55e0574897b9daff7104104b0f.parquet; error = 'Unsupported type override (Numeric -> Enum). Column DateCol will be parsed as Numeric'
  warnings.warn(w)


Symbol,Date,DateCol,feat_Open,feat_High,feat_Low,feat_Close,feat_Volume,feat_VROC11,feat_VROC7,feat_VROC5,feat_VROC3,feat_PROC11,feat_PROC7,feat_PROC2,feat_PROC3,feat_DOW,feat_RSIg7,feat_RSIg5,feat_RSIg3,feat_RSI17,feat_RSI13,feat_RSI11,feat_Close11/19SMARatio,feat_Close5/11SMARatio,feat_Close3/7SMARatio,feat_Close2PctFromSMA,feat_Close4PctFromSMA,feat_Close8PctFromSMA,feat_10CCI,feat_6CCI,feat_3CCI,feat_10FI,feat_6FI,feat_3FI,feat_10EMV,feat_6EMV,feat_3EMV,feat_Close10PctFromUpperBB,feat_Close10PctFromLowerBB,feat_Close10BBBandwidth,feat_Close10PctB,feat_Close6PctFromUpperBB,feat_Close6PctFromLowerBB,feat_Close6BBBandwidth,feat_Close6PctB,feat_Close3PctFromUpperBB,feat_Close3PctFromLowerBB,feat_Close3BBBandwidth,feat_Close3PctB,feat_DIRatio17,feat_ADX17,feat_DMI17,feat_AdjATR17,feat_DIRatio14,feat_ADX14,feat_DMI14,feat_AdjATR14,feat_DIRatio11,feat_ADX11,feat_DMI11,feat_AdjATR11,feat_DIRatio9,feat_ADX9,feat_DMI9,feat_AdjATR9,feat_DIRatio7,feat_ADX7,feat_DMI7,feat_AdjATR7,feat_DIRatio5,feat_ADX5,feat_DMI5,feat_AdjATR5,feat_DIRatio4,feat_ADX4,feat_DMI4,feat_AdjATR4,feat_DIRatio3,feat_ADX3,feat_DMI3,feat_AdjATR3,feat_DIRatio2,feat_ADX2,feat_DMI2,feat_AdjATR2,feat_Close91226MACD,feat_Close71123MACD,feat_235MACDg,feat_357MACDg,feat_5711MACDg,feat_11StocOsc,feat_11StocOscRatio,feat_7StocOsc,feat_7StocOscRatio,feat_5StocOsc,feat_5StocOscRatio,feat_3StocOsc,feat_3StocOscRatio,feat_11PriceChannelDist,feat_7PriceChannelDist,feat_5PriceChannelDist,feat_3PriceChannelDist,feat_PSAR,feat_10ADL,feat_10CMF,feat_10ChaikinOscillator,feat_5ADL,feat_5CMF,feat_5ChaikinOscillator,feat_10AroonUp,feat_10AroonDown,feat_10AroonOscillator,feat_5AroonUp,feat_5AroonDown,feat_5AroonOscillator,feat_autocorr10High1,feat_autocorr5High1,feat_autocorr5High2,feat_autocorr3High1,feat_autocorr3High2,feat_autocorr3High3,feat_autocorr2High2,feat_autocorr2High1,lab_lowPercentChange_H2,lab_percentChange_H2High,lab_expectedReturnPct_H2,weights
AAPL,2010-01-29,1.26472e+15,6.21,6.24,5.87,5.93,1.24595e+09,1.05639,1.03536,0.413017,-0.332684,-0.0414747,-0.0616541,-0.04,-0.0545455,5,28.5714,40.8602,17.9104,33.9535,35.7513,33.8983,-0.00624869,-0.00835469,-0.00556729,-0.0182119,-0.0458568,-0.0546034,-126.409,-108.512,-69.0777,0.0214999,0.0165793,0.00946995,-0.0113371,-0.0229038,-0.043297,-0.120238,0.00584483,13.3732,0.0407832,-0.0970525,0.0149558,11.6803,0.120565,-0.109277,0.0447835,15.9194,0.258921,-41.5142,44.924,-18.6498,0.0324032,-41.4171,43.9512,-18.2033,0.0358748,-40.6747,42.2358,-17.1793,0.0392435,-39.4963,40.4135,-15.9619,0.0440468,-37.6183,38.4104,-14.4493,0.0467407,-36.3066,39.7738,-14.4405,0.0490369,-38.1593,43.262,-16.5085,0.0525935,-46.313,52.3854,-24.2613,0.052448,-70.2081,73.9173,-51.8959,0.0525181,0.318865,0.272589,0.215742,0.213029,0.251032,7.69231,-0.710169,8.21918,-0.692354,8.21918,-0.716793,9.52381,-0.607913,-0.0527157,-0.0489174,-0.0489174,-0.0412288,-0.100866,-0.327443,-0.234136,0,-0.17178,-0.179354,0.0685022,30,100,-70,40,100,-60,0.462217,0.110884,-0.722802,0.965116,-0.730831,-0.592137,1,1,-0.00673401,0.020202,n0,1
AAPL,2010-02-01,1.26498e+15,5.94,6.05,5.9,6.01,7.49876e+08,0.73224,0.233036,-0.296353,-0.564675,-0.0692308,-0.0805471,-0.0457413,-0.0692308,1,34.5865,34.5238,20.2899,38.2075,39.6907,37.5691,-0.00706215,-0.0093068,-0.0120538,0.00670017,-0.0191758,-0.0318164,-102.339,-83.7009,-45.6555,0.0197697,0.0148737,0.0113073,-0.0124845,-0.016851,-0.0392331,-0.109107,0.0326528,14.7394,0.205208,-0.0865798,0.0361473,12.5902,0.269033,-0.0388169,0.0349062,7.38675,0.455099,-42.1464,45.132,-19.0215,0.0326214,-42.2515,44.4239,-18.7697,0.0361237,-42.037,43.3295,-18.2145,0.0407981,-41.6998,42.4309,-17.6936,0.0421162,-41.6801,42.2733,-17.6196,0.0461575,-44.6336,46.9446,-20.9531,0.0470335,-50.1768,53.2383,-26.7133,0.0455343,-62.5215,65.5577,-40.9877,0.0432765,-85.121,86.3574,-73.5083,0.0420441,0.290946,0.239662,0.0786703,0.147925,0.230613,17.9487,0.419929,19.1781,0.45003,19.1781,0.408124,29.7872,1.05116,-0.0399361,-0.0360866,-0.0360866,-0.015561,-0.0847101,-0.194253,-

In [ ]:
### HYPERPARAMETER TUNING AND VARIABLE SELECTION
features = [s for s in fullDF.columns if "feat_" in s]
#features = [s for s in fullDF.columns if "PC" in s]
labels = [s for s in fullDF.columns if "lab_" in s]

timePerRun = int(60*60*.5)

label = labels[0]
quantile_alpha=.05
perf_metric='mae'

print('building model for {}'.format(label))
# GBM hyperparameters
hyper_params = {'learn_rate': [i * 0.01 for i in range(1,21,2)],
                'max_depth': list(range(2, 11, 2)),
                'sample_rate': [i * 0.1 for i in range(5, 10)],
                'col_sample_rate': [i * 0.1 for i in range(3, 10)],
                'col_sample_rate_per_tree': [i * 0.1 for i in range(5, 8)],
                'min_rows': [50, 100, 500],
                'min_split_improvement': [1e-3, 1e-5]}
# Search criteria
search_criteria = {'strategy':'RandomDiscrete', 'max_models':1000, 'max_runtime_secs':timePerRun,
                    'stopping_metric':perf_metric, 'stopping_rounds':5, 'stopping_tolerance':0.0001}
gbm = H2OGradientBoostingEstimator(distribution="quantile", quantile_alpha = quantile_alpha, nfolds = 6, ntrees=500, learn_rate_annealing=0.99)
# Train and validate a random grid of GBMs
gbm_grid = H2OGridSearch(model=gbm,
                          grid_id='gbm_grid_{}'.format(label),
                          hyper_params=hyper_params,
                          search_criteria=search_criteria,parallelism=0)
gbm_grid.train(x=features,y=label,training_frame=fullDF,weights_column='weights')
gbm_gridperf=gbm_grid.get_grid(sort_by=perf_metric,decreasing=False,)

building model for lab_lowPercentChange_H2
gbm Grid Build progress: |████████████████████████████████████████████████

In [ ]:
resultingGrid = pandas.DataFrame(gbm_grid.sorted_metric_table()).drop('residual_deviance',axis=1)
modelScores=pandas.DataFrame(columns=['model_ids','mae','rmse'])
for i in range(len(gbm_gridperf.models)):
    mae=gbm_gridperf.models[i].mae(train=True,valid=True,xval=False)
    rmse=gbm_gridperf.models[i].rmse(train=True,valid=True,xval=False)
    modelScores.loc[-1] = [gbm_gridperf.models[i].model_id,mae['valid'],rmse['valid']]
    modelScores.index = modelScores.index + 1
    modelScores = modelScores.sort_index()
modelScores = modelScores.merge(resultingGrid,how='inner',on='model_ids')
modelScores = modelScores.sort_values('mae',ascending=True).reset_index(drop=True)

modelScores['trimmedModelId']=modelScores['model_ids'].apply(lambda x: x[:-5])
modelScores['col_sample_rate'] = modelScores['col_sample_rate'].astype('float64')
modelScores['col_sample_rate_per_tree'] = modelScores['col_sample_rate_per_tree'].astype('float64')
modelScores['learn_rate'] = modelScores['learn_rate'].astype('float64')
modelScores['max_depth'] = modelScores['max_depth'].astype('float64')
modelScores['min_rows'] = modelScores['min_rows'].astype('float64')
modelScores['min_split_improvement'] = modelScores['min_split_improvement'].astype('float64')
modelScores['sample_rate'] = modelScores['sample_rate'].astype('float64')

try: modelScores = modelScores.drop(columns=[''])
except: pass

modelScores[[x for x in modelScores.columns if x not in ['trimmedModelId','model_ids']]] = modelScores[[x for x in modelScores.columns if x not in ['trimmedModelId','model_ids']]].astype('float64')
groupedModelScores = modelScores.dropna(subset=['mae']).groupby('trimmedModelId').mean().sort_values('mae',ascending=True)

groupedModelScores['aggregate'] = (2 * groupedModelScores['mae'] + groupedModelScores['rmse'])/3
groupedModelScores = groupedModelScores.sort_values('aggregate',ascending=True)
groupedModelScores.head()

In [ ]:
featureImportance = gbm_gridperf.models['model_id'==list(modelScores[modelScores['trimmedModelId']==groupedModelScores.index[0]]['model_ids'])[0]].varimp(use_pandas=True)
topFeatures = list(featureImportance[featureImportance['percentage']>=.005]['variable'].values)
print(featureImportance.head(30))

In [ ]:
print(featureImportance.tail(20))

In [ ]:
### ITERATE THROUGH 4MO TRAINING + 1MO VALIDATION TO BUILD GBMS WITH CHECKPOINTING (USING HYPERPARAMETERS AND VARIABLES FROM ABOVE)

subsetMonths = [i for i in YrMos if i not in ootMonths]
subsetMonths = [subsetMonths[i:i + 5] for i in range(0, len(subsetMonths), 5)]
valMos=[]
rounds=1
for i in range(len(subsetMonths)):
    print('''Round {}'''.format(rounds))
    if len(subsetMonths[i])<3: break
    valMos.extend([subsetMonths[i][-1]])
    trainMos = subsetMonths[i][0:-2]
    trainFiles = ['YearMo='+str(f).split('YearMo=')[1] for f in Path(os.path.join(uyulala.dataDir,'transformed',folderName)).glob('**/*') if f.is_file() \
               & any([x in str(f) for x in ['YearMo=' + sub for sub in trainMos] ]) ]
    train = h2o.import_file(path=os.path.join(uyulala.dataDir,'transformed',folderName),pattern = "(%s)" % ('|'.join(trainFiles),),col_types={'DateCol':'enum','Date':'enum'}).na_omit().merge(h2o.import_file(path=os.path.join(uyulala.dataDir,'labeled',folderName),pattern = ".*\.parquet",col_types={'DateCol':'enum','Date':'enum'}).na_omit()).na_omit()
    validation = h2o.import_file(path=os.path.join(uyulala.dataDir,'transformed',folderName),pattern = "(%s)" % ('|'.join(valMos),),col_types={'DateCol':'enum','Date':'enum'}).na_omit().merge(h2o.import_file(path=os.path.join(uyulala.dataDir,'labeled',folderName),pattern = ".*\.parquet",col_types={'DateCol':'enum','Date':'enum'}).na_omit()).na_omit()
    if rounds == 1:
        gbm = H2OGradientBoostingEstimator(distribution="quantile", quantile_alpha = quantile_alpha, nfolds = 0, ntrees=int(100*rounds), 
                                          learn_rate_annealing=0.99,
                                          col_sample_rate=groupedModelScores['col_sample_rate'][0],
                                          col_sample_rate_per_tree=groupedModelScores['col_sample_rate_per_tree'][0],
                                          learn_rate = groupedModelScores['learn_rate'][0],
                                          max_depth=int(groupedModelScores['max_depth'][0]),
                                          min_rows=int(groupedModelScores['min_rows'][0]),
                                          min_split_improvement=groupedModelScores['min_split_improvement'][0],
                                          sample_rate=groupedModelScores['sample_rate'][0])
    else:
        gbm = H2OGradientBoostingEstimator(checkpoint = gbm.model_id, distribution="quantile", quantile_alpha = quantile_alpha, nfolds = 0, ntrees=int(100*rounds), 
                                          learn_rate_annealing=0.99,
                                          col_sample_rate=groupedModelScores['col_sample_rate'][0],
                                          col_sample_rate_per_tree=groupedModelScores['col_sample_rate_per_tree'][0],
                                          learn_rate = groupedModelScores['learn_rate'][0],
                                          max_depth=int(groupedModelScores['max_depth'][0]),
                                          min_rows=int(groupedModelScores['min_rows'][0]),
                                          min_split_improvement=groupedModelScores['min_split_improvement'][0],
                                          sample_rate=groupedModelScores['sample_rate'][0])
    gbm.train(x=features,y=label,training_frame=train,validation_frame=validation,weights_column='weights')
    print('''====================Round {} Performance on Validation Set===================='''.format(rounds))
    print('''MAE: {}'''.format(gbm.mae(train=False,valid=True,xval=False)))
    print('''RMSE: {}'''.format(gbm.rmse(train=False,valid=True,xval=False)))
    rounds=rounds+1

In [ ]:
train.shape